In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
from transformers import BertTokenizer, BertForSequenceClassification

from BERT_utils import *

# Import utility functions
sys.path.insert(0, r'c:\Users\joneh\master_thesis\src')
from main_utils import *

### Sentiment Analysis Testing

Sentiment analysis functions should take a panads series as input and output a pandas series of the same length with the sentiment of the input text. The sentiment should be a float between -1 and 1, where -1 is negative, 0 is neutral, and 1 is positive.

In [ ]:
# Load data
df = pd.read_csv(r'C:\Users\joneh\master_thesis\data\news\TheGuardian\TG_CrudeANDOil.csv')

# Load the pre-trained model
finbert = BertForSequenceClassification.from_pretrained(
    'yiyanghkust/finbert-tone',
    num_labels=3
)

# Load the pre-trained model's tokenizer
fb_tokenizer = BertTokenizer.from_pretrained(
    'yiyanghkust/finbert-tone'
)

labels: dict = {0:'neutral', 1:'positive', 2:'negative'}

In [ ]:
headlines = df['webTitle'].tolist()[:5]

display(headlines)

def get_BERT_sentiment(sentence_list: list[str], model=finbert, tokenizer=fb_tokenizer):
    
    inputs = tokenizer(
        sentence_list, 
        return_tensors="pt", 
        padding=True
    )

    outputs = finbert(**inputs)[0]

    res = {}

    for i, sentence in enumerate(sentence_list):
        res[i] = [np.argmax(outputs.detach().numpy()[i]), sentence]

    df = pd.DataFrame.from_dict(
        res, 
        orient='index', 
        columns=['prediction', 'sentence']
    )

    compounded_score = df.prediction.value_counts(normalize=True).to_dict()

    return compounded_score

print(get_sentiment(headlines))